# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# query 400 rows of data query data from table
select_query = "select * \
from news_national_medol gr \
where id not in (select news_national_medol_id from news_national_medol_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [6]:
ds_title = source_ds
res_ds = process_by_df(ds_title,'title')

  0%|          | 0/139 [00:00<?, ?it/s]

  1%|          | 1/139 [00:00<00:42,  3.24it/s]

  1%|▏         | 2/139 [00:00<00:42,  3.20it/s]

  2%|▏         | 3/139 [00:01<01:05,  2.08it/s]

  3%|▎         | 4/139 [00:01<01:14,  1.81it/s]

...

 96%|█████████▌| 133/139 [00:13<00:00, 13.50it/s]

 97%|█████████▋| 135/139 [00:13<00:00, 14.28it/s]

 99%|█████████▊| 137/139 [00:13<00:00, 14.10it/s]

100%|██████████| 139/139 [00:13<00:00, 11.86it/s]

100%|██████████| 139/139 [00:13<00:00, 10.13it/s]

In [7]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,publish_date_convert,calculation_mark,stemmed,tokenized_tweet,Prediction,Score
0,67406,"Kasus Vina, Pihak Pegi Setiawan Masukkan 3 Sur...",Siti Yona Hukmana • 05 Juni 2024 23:20\n \n \...,05 Juni 2024 23:20,,https://www.tribunnews.com/,https://www.medcom.id/nasional/hukum/1bV9RaLN-...,2024-06-05 23:40:27.072556,7,True,,Siti Yona Hukmana,False,2024-06-05 23:20:00,1,kasus vina pihak pegi setiawan masukkan surat ...,"{vina, surat, perkara, gelar, setiawan, pegi, ...",Neutral,0.999744
1,67405,Polisi Tetapkan Majikan ART yang Meninggal Lom...,Hendrik Simorangkir • 05 Juni 2024 23:12\n \n ...,05 Juni 2024 23:12,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/4baaB1Wb...,2024-06-05 23:40:27.061212,7,False,,Hendrik Simorangkir,False,2024-06-05 23:12:00,1,polisi tetapkan majikan art yang meninggal lom...,"{art, meninggal, lompat, tersangka, polisi, la...",Neutral,0.999640
2,67404,90 Ribu Pelanggan Terdampak Imbas Pipa PDAM Ti...,Roni Kurniawan • 05 Juni 2024 23:03\n \n \n ...,05 Juni 2024 23:03,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/0KvP5Zpb...,2024-06-05 23:40:27.054306,7,False,,Roni Kurniawan,False,2024-06-05 23:03:00,1,ribu pelanggan terdampak imbas pipa pdam tirta...,"{pecah, kota, imbas, terdampak, pipa, bandung,...",Neutral,0.999443
3,67403,"Pencarian Hari Kedua, 1 Penambang Pasir Korban...",Daviq Umar Al Faruq • 05 Juni 2024 22:56\n \n ...,05 Juni 2024 22:56,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/8koP61DK...,2024-06-05 23:40:27.048280,7,False,,Daviq Umar Al Faruq,False,2024-06-05 22:56:00,1,pencarian hari kedua penambang pasir korban te...,"{pasir, ditemukan, penambang, longsor, korban,...",Neutral,0.999668
4,67402,ART yang Meninggal Lompat dari Rumah Majikan A...,Hendrik Simorangkir • 05 Juni 2024 22:51\n \n ...,05 Juni 2024 22:51,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/3NO1Mx2k...,2024-06-05 23:40:27.041202,7,False,,Hendrik Simorangkir,False,2024-06-05 22:51:00,1,art yang meninggal lompat dari rumah majikan a...,"{art, meninggal, lompat, rumah, memar, paru, m...",Neutral,0.994836


In [8]:
# record result
sentiment_class = 0

# INSERT INTO public.news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES( 0, 0, 0);

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "INSERT INTO news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES(%s, %s, %s);" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


...

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


In [9]:
ds_content = source_ds
res_ds_content = process_by_df(ds_content,'content')

  0%|          | 0/139 [00:00<?, ?it/s]

  1%|          | 1/139 [00:00<00:48,  2.86it/s]

  1%|▏         | 2/139 [00:00<00:47,  2.88it/s]

  2%|▏         | 3/139 [00:01<00:45,  3.01it/s]

  3%|▎         | 4/139 [00:01<00:45,  2.99it/s]

...

 98%|█████████▊| 136/139 [00:49<00:00,  3.43it/s]

 99%|█████████▊| 137/139 [00:49<00:00,  3.25it/s]

 99%|█████████▉| 138/139 [00:50<00:00,  3.35it/s]

100%|██████████| 139/139 [00:50<00:00,  3.09it/s]

100%|██████████| 139/139 [00:50<00:00,  2.75it/s]

In [10]:
res_ds_content.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,publish_date_convert,calculation_mark,stemmed,tokenized_tweet,Prediction,Score
0,67406,"Kasus Vina, Pihak Pegi Setiawan Masukkan 3 Sur...",Siti Yona Hukmana • 05 Juni 2024 23:20\n \n \...,05 Juni 2024 23:20,,https://www.tribunnews.com/,https://www.medcom.id/nasional/hukum/1bV9RaLN-...,2024-06-05 23:40:27.072556,7,True,,Siti Yona Hukmana,False,2024-06-05 23:20:00,1,siti yona hukmana juni jakarta kuasa hukum peg...,"{smash, ciri, tersangka, juni, hukmana, vina, ...",Neutral,0.999182
1,67405,Polisi Tetapkan Majikan ART yang Meninggal Lom...,Hendrik Simorangkir • 05 Juni 2024 23:12\n \n ...,05 Juni 2024 23:12,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/4baaB1Wb...,2024-06-05 23:40:27.061212,7,False,,Hendrik Simorangkir,False,2024-06-05 23:12:00,1,hendrik simorangkir juni tangerang polres metr...,"{dwi, rumah, rp, j, tersangka, juni, uang, tew...",Neutral,0.993174
2,67404,90 Ribu Pelanggan Terdampak Imbas Pipa PDAM Ti...,Roni Kurniawan • 05 Juni 2024 23:03\n \n \n ...,05 Juni 2024 23:03,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/0KvP5Zpb...,2024-06-05 23:40:27.054306,7,False,,Roni Kurniawan,False,2024-06-05 23:03:00,1,roni kurniawan juni bandung terdapat sekitar r...,"{pecahnya, batununggal, penjabat, bengkok, hil...",Neutral,0.986633
3,67403,"Pencarian Hari Kedua, 1 Penambang Pasir Korban...",Daviq Umar Al Faruq • 05 Juni 2024 22:56\n \n ...,05 Juni 2024 22:56,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/8koP61DK...,2024-06-05 23:40:27.048280,7,False,,Daviq Umar Al Faruq,False,2024-06-05 22:56:00,1,daviq umar al faruq juni lumajang seorang pena...,"{pasir, dusun, selasa, longsor, dwi, kusnadi, ...",Neutral,0.999350
4,67402,ART yang Meninggal Lompat dari Rumah Majikan A...,Hendrik Simorangkir • 05 Juni 2024 22:51\n \n ...,05 Juni 2024 22:51,,https://www.tribunnews.com/,https://www.medcom.id/nasional/daerah/3NO1Mx2k...,2024-06-05 23:40:27.041202,7,False,,Hendrik Simorangkir,False,2024-06-05 22:51:00,1,hendrik simorangkir juni tangerang cicih asist...,"{rumah, dimandikan, juni, pemakaman, kematian,...",Neutral,0.980324


In [11]:
# record result
sentiment_class = 0

for index, row in res_ds_content.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "update news_national_medol_sentiment set sentiment_category_content = %s, score_content = %s where news_national_medol_id = %s" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


...

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
